# Introductory Statistics Workshop

Welcome to the workshop! Today you will be learning about:

1. Causation, Correlation, Different types of studies, and Random Sampling
2. Hypothesis Testing using p-values
3. Histograms
4. Confidence Intervals
5. SD, Mean, and the Central Limit Theorem

Throughout this notebook we use a python library called **Pandas** in order to store and display our data. If you would like to learn how to use Pandas in more detail, check out our Python Libraries workshop materials: http://bit.ly/2oY6BOP

https://pypi.python.org/pypi/datascience/

In [0]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

# **About the dataset:**

In 2013, students of the Statistics class at FSEV UK were asked to invite their friends to participate in this survey.

The data file (responses.csv) consists of 1010 rows and 150 columns (139 integer and 11 categorical).

Each row represents a young person's response. Each column is a question/topic on which each person had to give a rating or information. This link has more detailed description of the dataset: [Young People Survey](https://www.kaggle.com/miroslavsabo/young-people-survey).

Here is an example of convenience sampling, which isn't as accurate as pure simple random sampling, but is still reliable enough to formulate valid conclusions from. You are making conclusions of this data and not necessarily trying to generalize the conclusions to the whole population of young people, so convenience sampling is ok. Many times in real life, when conducting observational studies, you will be working with data that came from convenience sampling.



In [0]:
#Your code could look like

# Import data :
#data_all = pd.read_csv('DSS 4-4 Stat Workshop/responses.csv')
#or
#data_all = pd.read_csv('folder_name/responses.csv')



columns = pd.read_csv("https://raw.githubusercontent.com/krutikaingale/Data-Science-Society/master/columns.csv")
data_all = pd.read_csv("https://raw.githubusercontent.com/krutikaingale/Data-Science-Society/master/responses.csv")

In [0]:
data_all

**Now let's study spending habits, specifically those of males and females**

In [0]:
#We pick the columns that relate to spending, and pick the gender column
#Columns are indexed at 0. So we can reference columns by their index 0,1,2,...149
spending  = data_all.iloc[:,134:140] #.iloc[:, a:b] chooses columns indexed at a up to but not including b
spending['Gender'] = data_all['Gender']
spending

**The columns represent these questions:**

1) I enjoy going to large shopping centres.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

2) I prefer branded clothing to non branded.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

3) I spend a lot of money on partying and socializing.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

4) I spend a lot of money on my appearance.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

5) I spend a lot of money on gadgets.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

6) I will hapilly pay more money for good, quality or healthy food.: Strongly disagree 1-2-3-4-5 Strongly agree (integer)

**From now on, when we reference "male and female spending", we mean "male and female spending ratings"**

In [0]:
#We can tell entire dataframe to drop NaN values and convert them to 0's.
spending.fillna(0, inplace = True)
spending.head()

***Group Question***: What do larger ratings mean for spending habits? What do smaller ratings mean for spending habits? How can we combine all the responses for all 6 questions into 1 informative response?

**In the table, larger numbers/ratings in essence implies that the person spends more. So let's sum up all the spending of each person into a single spending column. The resulting summed column will be of type float.**

In [0]:
#Basically larger numbers/ratings means the person spends more
#So let's sum up all the spending of each person into a single spending column. Resulting column will be of float type
spending_amount = spending.iloc[:, 0] + spending.iloc[:, 1] + spending.iloc[:, 2] + spending.iloc[:, 3] + spending.iloc[:, 4] + spending.iloc[:, 5] 
total_df = pd.DataFrame(spending_amount).rename(columns = {0: 'Spending Amount'})
total_df['Gender'] = spending['Gender']
total_df

**Now let's create a table with female spending and a table for male spending.**

In [0]:
#female spending data frame
female_spending = total_df[total_df['Gender'] == 'female']
female_spending

In [0]:
#male spending data frame
male_spending = total_df[total_df['Gender'] == 'male']
male_spending

***Group Question***: Now which simulation method should we use (shuffling, bootsrapping, etc) and why? Could you give a basic outline of the simulation process?

**Now we want to stack the total spending of females on top of the spending of males and put it into a data frame. We do this so we can run our simulation of shuffling for our hypothesis test.**

In [0]:
appended_spending = female_spending['Spending Amount'].append(male_spending['Spending Amount'], ignore_index=True)

print("Number of Females: ", len(female_spending['Spending Amount']))
print("Number of Males: ",len(male_spending['Spending Amount']))
#These are only females and males, There are people who idenitfy otherwise in the actual dataset, but we're only focused on females and males
final_spending = pd.DataFrame(appended_spending)
final_spending
#females are the first 593 rows, males are the next 411 rows

The females are the first 593 rows, males are the next 411 rows. There are 1004 rows in total.

***Group Question***: What is a good test statistic to use and why? Remember we're trying to compare two groups.

**Let's compare the spending of males and females. We can do this by subtracting the male and female average spending. This will serve as our test statistic : male average spending - female average spending.**

In [0]:
observed_test_statistic = male_spending['Spending Amount'].mean() - female_spending['Spending Amount'].mean()
observed_test_statistic


**The positive difference of 0.85 indicates that the male spending seems to be higher than the female spending, and that females may be more money-efficient than males. Higher ratings from the males implies that males don't mind spending more money.**

#Let's start our Hypothesis Test

***Group Question***: What is our null and alternative hypothesis?

**Null Hypothesis**: There is no difference between male and female spending. The females' spendings are like a random sample of 593  out of all 1004 expenditures. If the males' average spending came out higher than that of the females, it is due to chance variation.

**Alternative Hypothesis**: The males' spending being higher than the females' spending is not due to chance. There is something other than chance causing the difference between male and female spending.

When you sample without replacement (by specifiying with_replacement=False) the same number of times as there are rows, you end up with a random shuffle of all the rows, which represents finding a new sample. Under the null hypothesis, differences in the female and male spending averages should be close to 0 (aka there is no difference between male and female spending). Under the null hypothesis, each new sample we get from shuffling the rows, male spending average - female spending average should be close to 0.

**Overarching method**: We shuffle the rows of final_spending by random sampling. Choose the top 593 rows of resulting sampled dataframe to represent female spendings and find the average spending. The remaining 411 rows will represent male spending and find the average. 

**Our test statistic: male spending average - female spending average.**

Pseudocode:


```
Declare an test_statistics_array

Repeat process 10,000 times to get 10,000 test statistics:

   In one sample:
   
           We shuffle the rows of final_spending by random sampling. Choose the top 593 rows of resulting sampled dataframe 
           to represent female spendings and find the average spending. The remaining 411 rows will represent male spending 
           and find the average. 
           
           this_sample's_test_statistic = male spending average - female spending average
           
           test_statistics_array.add on (this_sample's_test_statistic)
```


In [0]:
simulated_statistics = []
repetitions = 10000

for i in np.arange(repetitions):
    shuffled = final_spending.sample(n = len(final_spending.index), replace=False)
    female_mean = (shuffled.iloc[0:593])['Spending Amount'].mean()
    male_mean = (shuffled.iloc[593:len(final_spending.index)])['Spending Amount'].mean()
    test_statistic = male_mean - female_mean
    simulated_statistics = np.append(simulated_statistics, test_statistic)

In [0]:
simulated_statistics

In [0]:
len(simulated_statistics)

Our observed test statistic was 0.8549870139461575.

**To calculate our p-value, we do empirical_P = np.count_nonzero(simulated_statistics >= observed_statistic)/repetitions**

**P-value** : Chance (computed under the null hypothesis) of getting a test statistic equal to the one that was observed or more in the direction of the alternative. In our case, "direction of the alternative" means greater than or equal to 0.8549870139461575.

**P-value** = (how many differences that we calculated under the null hypothesis are greater than what was observed in the beginning)/ total number of differences. By dividing we get the "chance" that we're looking for. We're trying to see if the observed test statistic is too extreme.


In [0]:
empirical_P = np.count_nonzero(simulated_statistics >= observed_test_statistic)/repetitions
print('Observed Statistic:', observed_test_statistic)
print('Empirical P:', empirical_P)

results = pd.DataFrame(np.array(simulated_statistics)).rename(columns = {0 : 'Simulated Statistics'})
results['Simulated Statistics'].plot(normed = True, kind='hist', color = 'navy')
plt.xlabel('Difference in Spending (M-F)')
plt.ylabel('Proportion')
plt.title('Histogram of Differences Scores')

plt.scatter(observed_test_statistic, 0, color='red', s=50);
left = results['Simulated Statistics'].sort_values(ascending = True).quantile(0.025)
right = results['Simulated Statistics'].sort_values(ascending = True).quantile(0.975)
print('95% Confidence Interval:', left, ", ", right )
plt.plot((left, right), (0, 0), 'yellow', lw = 8)
plt.show()

***Group Question***: What is the conclusion of our test and why? Explain.

P VALUE = .0019

**Under null hypothesis, male spending - female spending should be close to 0, since we assume male and female spending is the same. The above histogram does show that, since it is centered at 0.**


However, we see the observed test statistic is very "extreme" and lies away from the heart of the distribution. **The observed test statistic, in red, is about 0.85, which means the male spending avergae - female spending average is positive. This suggests that the males spent more money than the females and that the females save more money.**

**The p-value is 0.0019 and we use a p-value cutoff of 0.05 along with a 95% confidence interval.** 
Since **0.0019 < 0.05, and even < 0.01**,  we see that the **difference between male and female spending that was observed in the beginning is too extreme to have occurred by just chance**. There is something other than chance that caused the males to spend more than females.

**So we reject the null hypothesis that male and female spending habits of the young people who took the survey are the same.**

# Visualizing Data

## 1. Bar Charts

In [0]:
spending.head()

We can analyze some of this data using a bar graph.

### Question: What does the data tell us about people's willingness to spend more money on "good" food?
### Answer: Use a bar graph - what are the categorical variables we'll use?

<br>
Notes:
- can rearrange bars
- can flip the orientation of the graph

In [0]:
grouped = spending.groupby('Spending on healthy eating').count().iloc[:, :1]
grouped.index = ["", 'Strongly Disagree', 'Disagree', 'Neutral', "Agree", "Strongly Agree"]
grouped.index.names = ['Willing to spend more on food']
grouped.columns = ['responses']

# grouped = grouped.sample(6, replace=False) # We can rearrange bars and still read the graph

grouped.plot(kind="barh")   # barh to flip bar

## 2. Histograms

**Data:** Top 200 highest grossing movies of all time.

In [0]:
import matplotlib.pyplot as plt
movies = pd.read_csv("https://raw.githubusercontent.com/carlocrza/Data_Science_Society/master/top_movies.csv")
movies.head()

In [0]:
movies.to_csv("testing.csv")

g = movies[['Studio', 'Title']].groupby("Studio").count()
plt.barh(g.index, g['Title'])

Say we wanted to know how many movies made more than `$`500 billion and how many made more than `$`600 million, etc. This is a distribution of the movie gross. We use a histogram.


First, let's think about our x-axis: it will represent the amount of money made for movies. 

In [0]:
gross = movies[['Gross (Adjusted)']].apply(lambda x: np.round(x/1e6, 2))['Gross (Adjusted)']

min(gross), max(gross)

Okay so our x-axis will go from `$`338 to `$`1796. Let's divide this range into **bins of width 100**

In [0]:
np.histogram(gross, bins=np.arange(300,2001,100))

In [0]:
gross.hist(bins=np.arange(300,2001,100), figsize=(15,7))

Y-axis above is the number of occurrences.

Can have unequal bin widths. Sometimes this is useful.

But problem: now it looks like there are a ton of movies that make more than 700 million.

In [0]:
gross.hist(bins=list(np.arange(300,700,100)) + [700,2000], figsize=(15,7))

In [0]:
gross.hist(bins=list(np.arange(300,700,100)) + [700,2000], figsize=(15,7), density=True)

Now the y-axis represents "DENSITY" of Data. The height for the bin \[700, 2000) is now very low because it has super-low density (think huge bin width but few entries). Compared to the bin 300-400 with only a width of 100 but ~70 entries. 

How is this useful?
- Visually makes more sense: even with unequal bins, we can see that movies from 300-400 are much more common than movies above 700
- The area of a bin = the percent of values that appear in that bin

$\mbox{area of bar} ~=~ \mbox{percent of entries in bin}\\
\mbox{area of bar} ~=~ \mbox{height of bar} \times \mbox{width of bin}$

### QUESTION: What percent of movies make between `$`300 and `$`500 million dollars?

In [0]:
# not yet




















100*.0034 + 100*.003

## Types of Distributions

<table>
  
  <tr><td>SYMMETRIC:<br> Amount of data to the left and right of the mean are roughly equal.</td><td>SKEWED RIGHT: <br>
1. tail of distribution is on the right hand side<br>
  2. mean > median</td><td>SKEWED LEFT: <br>
1. tail of distribution is on the left hand side<br>
  2. mean &lt; median</td>
  
  <tr><td>

<img src="https://www.siyavula.com/read/maths/grade-11/statistics/images/7588d6507c1315bd04332b326df7bec5.png"></td>

<td>

<img src="https://www.siyavula.com/read/maths/grade-11/statistics/images/690b5a06e1e448c583a88be23f6c747f.png"></td>

<td>

<img src="https://github.com/carlocrza/Data_Science_Society/blob/master/left_skewed.png?raw=true" width="450"></td>
</tr>
</table>

## Question: Is our movies distribution symmetric, right, or left skewed?

In [0]:
gross.hist(bins=list(np.arange(300,2000,100)), figsize=(15,7), density=True)

In [0]:
gross.hist(bins=list(np.arange(300,2000,100)), figsize=(15,7), density=True)
plt.axvline(np.mean(gross), color="red")
plt.axvline(np.median(gross), color="orange")

In [0]:
np.mean(gross), np.median(gross)

# Standard Deviation

Root mean square of deviations from average.

Compute the Standard Deviation for our Adjusted Gross data from movies. That is, what is the SD of the data we plotted a histogram of?

In [0]:
np.mean(gross)

In [0]:
deviations = gross - np.mean(gross)
deviations.head()

In [0]:
squared_deviations = deviations**2
squared_deviations.head()

In [0]:
mean_squared_deviations = np.mean(squared_deviations)

standard_deviation = np.sqrt(mean_squared_deviations)

standard_deviation

In [0]:
# hint for the future ;)
np.std(gross)

In [0]:
gross.hist(bins=list(np.arange(300,2000,100)), figsize=(15,7), density=True)
plt.axvline(np.mean(gross), color="red")
plt.plot((np.mean(gross), np.mean(gross)+np.std(gross)), (0, 0), 'yellow', lw = 8)

Let's see Standard Deviation on a symmetric (normal) distribution.

The heights of people tend to be symmetrically distributed. Let's look at heights of mothers from a sample.

In [0]:
sample = pd.read_csv("https://raw.githubusercontent.com/carlocrza/Data_Science_Society/master/baby.csv")
heights = sample['Maternal Height']
heights.head()

In [0]:
import seaborn as sns
sns.distplot(heights, bins=20, kde=False) # set kde=True to see curvature

plt.axvline(np.mean(heights), color="red")
plt.axvline(np.median(heights), color="orange")

plt.axvline(np.mean(heights) + 2*np.std(heights), color="green")
plt.axvline(np.mean(heights) - 2*np.std(heights), color="green")

# Central Limit Theorem

### The distribution of the sum or average of a sample drawn with replacement will be roughly symmetric

- use large random sample
- will always work! regardless of original sample distribution

For example, let's go back to our Gross returns for our movies data.

This was the original distribution:

In [0]:
gross.head()

In [0]:
gross.hist(bins=list(np.arange(300,2000,100)), figsize=(15,7), density=True) # switch to 10, 3

Clearly it's not a normal distribution as we saw before. But if we continuously draw samples with replacement from the data and take their average or sum, we'll get a symmetric distribution!

In [0]:
#original data
gross

repetitions = 10000
sample_size = 30
means = np.array([])

for i in np.arange(repetitions):
  sample = gross.sample(sample_size)
  new_mean = np.mean(sample)
  means = np.append(means, new_mean)

means = pd.Series(means)

In [0]:
means.hist(bins=20, figsize=(15,7), density=True)

# plt.axvline(np.mean(means), color="red")
# plt.axvline(np.median(means), color="orange")

# plt.axvline(np.mean(means) + np.std(means), color="green")
# plt.axvline(np.mean(means) - np.std(means), color="green")

In [0]:
np.mean(gross)

Allows us to answer questions about our original data?

Say we didn't have our original data and only samples of it, this allows us to answer what is the average gross return of the movies. Even without ever calling np.mean on the data.

Useful if you think about huge theoretical datasets that can't exist - like the US population. You can't record every single person and then take the mean, you can only take samples of the US population. However, if you take enough samples you can create this normal distribution and the mean of the normal distribution is the mean of the original population.

Real world example: suppose we are trying to determine how an election will turn out. We take a poll and find that in our sample, 80% of people would vote for candidate A over candidate B. Of course, we have only observed a small sample of the overall population. What percent of people in the whole population want to vote for candidate A? The central limit theorem tells us that if we ran the poll over and over again, the resulting guesses would be normally distributed around the true population value.

* Note that this works because % is considered a 'sum'

# Correlation

## Correlation does not equal Causation

Consider the scatter plot below which plots Amazon stock prices against Coca Cola stock prices? As you can see they are positively correlated - as Amazon stock increases, Coke stock increases. Does that mean better performance by amazon causes Coke to do better?

In [0]:
stocks = pd.read_csv("https://raw.githubusercontent.com/carlocrza/Data_Science_Society/master/stock.csv")
stocks['in'] = stocks.index
plt.scatter(stocks['AMZN Closing Price'], stocks['KO Closing Price'], s=4)
plt.xlabel("Amazon Stock")
plt.ylabel("Coca Cola Stock")

In [0]:
plt.scatter(stocks.index, stocks['KO Closing Price'], s=4)
plt.xlabel("Date")
plt.ylabel("Coca Cola Stock")
plt.plot(np.unique(stocks.index), np.poly1d(np.polyfit(stocks.index, stocks['KO Closing Price'], 1))(np.unique(stocks.index)))

In [0]:
# let's find the linear function manually

def error(slope_and_intercept):
    slope = slope_and_intercept[0]
    intercept = slope_and_intercept[1]
    x = stocks['in']
    y = stocks['KO Closing Price']
    fitted = slope*x + intercept
    return np.mean((y - fitted) ** 2)
  
error([1, 0])

In [0]:
from scipy.optimize import minimize
minimize(error, [1, 0])

In [0]:
def equation(x):
  return 1.09444470e-02 * x + 1.27460402e+01
  
x = np.arange(2500)
plt.plot(x, equation(x))

### Please [leave feedback here](https://docs.google.com/forms/d/e/1FAIpQLSfoDgUzuvkkiq4lR5MvodDK0ZLr7aCv9DA3yUJpnFSiu1i3tg/viewform?usp=sf_link). Thank you!!